# LAB: Utilizando Pandas con datos de oferta de departamentos en la CABA

## Introducción

El objetivo de este LAB es trabajar con los conceptos de agregación a través de `groupby` y `pivot tables` mencionadas en la sección teórica. 

Para eso usaremos un dataset elaborado por el Gobierno de la Ciudad de Buenos Aires que releva el 

> [p]recio de oferta de venta de terrenos de la Ciudad Autónoma de Buenos Aires, representado en puntos de valor del suelo correspondientes a cada parcela. Con el fin de analizar el precio de oferta, se toma como unidad de referencia el precio del metro cuadrado (m2). Se utiliza el precio en dólares, puesto que ésta es la moneda que se usa en el mercado como referencia para este tipo de transacciones. Otras variables consideradas son las dimensiones, ubicación geográfica, y la normativa del Código de Planeamiento Urbano que rige la zona en que el terreno analizado se localiza. 

Pueden descargar el dataset y otros elaborados para los años 2001-2015 desde este [link](
https://data.buenosaires.gob.ar/dataset/departamentos-en-venta). En el pdf adjunto pueden encontrar una descripción sucinta de las variables relevadas.

Importamos los paquetes necesarios y cargamos los datos.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks")

In [ ]:
df = pd.read_csv("./Data/ED_deptos_venta_2014.csv",encoding = "latin1", sep=";")

In [ ]:
# Veamos los tipos de las columnas...
df.dtypes

In [ ]:
# Revisamos que los datos se han importado correctamente

df.head()

### 1. ¿Cuál es el valor promedio del metro cuadrado en la CABA?

In [ ]:
df['U_S_M2'].mean()

### 2. ¿Cómo es la distribución cuartílica de la antigüedad de las viviendas de la CABA?

In [ ]:
df['ANTIGUEDAD'].quantile(q=[0.25, 0.5, 0.75])

### 3. Comparar las distribuciones decílicas de la antigüedad y precios de la viviendas en CABA?

In [ ]:
deciles = np.arange(0.1, 1.1, 0.1)

pd.concat([pd.Series(df['ANTIGUEDAD'].quantile(q=deciles)),
          pd.Series(df['U_S_M2'].quantile(q=deciles))], axis=1)

### 4. ¿Cuál es la media de precio por metro cuadrado para cada barrio de la CABA? ¿Cuál es barrio más caro?
Nota: realizar el cálculo tanto con groupby como con pivot tables

In [ ]:
df.pivot_table('U_S_M2', index='BARRIO', aggfunc=np.mean).sort_values(by='U_S_M2',ascending=False)

In [ ]:
df.groupby('BARRIO')['U_S_M2'].mean().sort_values(ascending = False)

### 5. ¿En qué barrio hay una mayor dispersión del precio por metro cuadrado (`U_S_M2`)?

**Pistas**
* Una primera opción es definir una función con arrays y utilizar `.apply()`

In [ ]:
def coef(x):
    x = np.array(x)
    return(np.std(x, ddof=1)/np.mean(x))

In [ ]:
df.groupby('BARRIO')['U_S_M2'].apply(coef).sort_values(ascending = False)

* Una segunda, es generar dos series: 
  - una con el método `.std()` y dividirla por otra serie generada con `mean()`

In [ ]:
(df.groupby('BARRIO')['U_S_M2'].std()/df.groupby('BARRIO')['U_S_M2'].mean()).sort_values(ascending = False)

**Bonus:** ¿cuál le parece mejor y por qué?

> En términos de performance, parece que la segunda opción es más rápida. En cambio, pareciera que la primera es notablemente más legible.

### 6. ¿En qué barrio los departamentos son más grandes? 
Nota: se puede realizar el cálculo tanto con groupby como con pivot tables

In [ ]:
df.pivot_table('M2', index='BARRIO', aggfunc=np.median).sort_values(by='M2', ascending=False)

In [ ]:
df.groupby('BARRIO')['M2'].median().sort_values(ascending=False)

### 7. En general, puede observarse alguna diferencia entre el precio mediano de aquellos departamentos que tienen terraza y los que no? ¿Y en el tamaño?

In [ ]:
df.groupby('TERRAZA')['U_S_M2'].median()

In [ ]:
df.groupby('TERRAZA')['M2'].median()

### 8. Genere un `DataFrame` que agregue la información del precio por M2 (`U_S_M2`), ambientes (`AMBIENTES`) y antigüedad (`ANTIGUEDAD`) a nivel de `COMUNA` y barrio (`BARRIO`). Proporcione información tanto de la tendencia central como de la dispersión de ambas distribuciones.

In [ ]:
df.pivot_table(['U_S_M2','AMBIENTES','ANTIGUEDAD'],index=['COMUNA','BARRIO']
               , aggfunc={'U_S_M2':[np.mean,np.std,len],
                          'AMBIENTES': [np.mean,np.std],
                          'ANTIGUEDAD': [np.mean,np.std]})

### 9. Si tuvieran que tomar una decisión sobre dónde invertir en una casa... ¿qué barrio elegirían? ¿Por qué?